# Project Notebook: Optimizing DataFrames and Processing in Chunks

project 2

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [ ]:
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Your code goes here
memory_footprints = []
chunk_iter = pd.read_csv("https://bit.ly/3H2XVgC", nrows = 5)
print(chunk_iter)


Reading 1000 and calculating total  memory 

In [3]:
chunk = chunk_iter.dropna()

In [ ]:
memory_footprints = []
chunk_iter = pd.read_csv("https://bit.ly/3H2XVgC", nrows = 1000)
print(chunk_iter)

In [ ]:
chunk_iter = pd.read_csv("https://bit.ly/3H2XVgC")
chunk_iter.info(memory_usage = "deep")

## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

How many columns have a numeric type?

In [ ]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
q = ['float64']

for chunk in chunk_iter:
  chunk_iter_q = chunk.select_dtypes(include=q)
  print(len(chunk_iter_q.columns))


How many have a string type?

In [ ]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for chunk in chunk_iter:
  str_type = chunk.select_dtypes(include=['object']).columns
  print(len(str_type))

Calculate the total memory usage across all of the chunks.

In [ ]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for N in chunk_iter:
  print(N.info(memory_usage="deep"))

How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?

In [ ]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for val in chunk_iter:
  for col in val.select_dtypes(include=['object']):
    print(f'{col}',len(val[col].unique()))

## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [ ]:
# Your code goes here


Determine which string columns you can convert to a numeric type if you clean them. For example, the int_rate column is only a string because of the % sign at the end.

In [ ]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for NUMERIC in chunk_iter:

  NUMERIC['int_rate'] = NUMERIC['int_rate'].str.replace('%', '')
 

  cols = ['int_rate']
  for col in cols:
    NUMERIC[col] = NUMERIC[col].astype('float')
print(NUMERIC)

Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and sub_grade columns. Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [ ]:
chunk_iter  = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for U in chunk_iter :
  for col in U.select_dtypes(include=['object']):
    sum = len(U[col])
    U_sum = len(U[col].unique())
    Q = (U_sum/sum)*100
    if Q < 50:
      U[col] = U[col].astype('category')
print(U)

## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [ ]:
import pandas as pd

chunk_iter = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for q in chunk_iter:
  q_columns = q.select_dtypes(include=['float64'])
  num = q_columns.columns[q_columns.isna().any()].tolist()
  q[num] = q[num].fillna(0)
  storage = (q.memory_usage(deep=True).sum())
  print(q.info(memory_usage="deep"))

## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [7]:
# Your code goes here
import pandas as pd

chunk_iter = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
